#### Import

In [ ]:
import easyocr
import os
import torch
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import os
import random
import glob
from tqdm import tqdm
import numpy as np
import time
from mmocr_eval import eval_ocr_metric
import re
import cv2 as cv
import onnxruntime
from torchvision import transforms

In [ ]:
# !pip install onnx
# !pip a onnxruntime-gpu
# !pip install icecream
# !pip install matplotlib
# !pip install rapidfuzz
# !pip install ipywidgets

In [ ]:
torch.cuda.is_available()

In [ ]:
# https://www.kaggle.com/code/fulrose/how-to-apply-new-font-to-matplotlib-easily/notebook
def change_matplotlib_font(font_download_url):
    FONT_PATH = 'MY_FONT'

    # font_download_cmd = f"wget {font_download_url} -O {FONT_PATH}.zip"
    # unzip_cmd = f"unzip -o {FONT_PATH}.zip -d {FONT_PATH}"
    # os.system(font_download_cmd)
    # os.system(unzip_cmd)


    font_files = fm.findSystemFonts(fontpaths=FONT_PATH)
    for font_file in font_files:
        fm.fontManager.addfont(font_file)

    try:
        # print(font_files)
        font_name = fm.FontProperties(fname=font_files[0]).get_name()
    except:
        print('Font not found')
        font_name = 'Kanit'
    matplotlib.rc('font', family=font_name)
    print("font family: ", plt.rcParams['font.family'])
font_download_url = 'https://fonts.google.com/download?family=Kanit'
change_matplotlib_font(font_download_url)

### Deployment

##### Load pth model

In [ ]:
# model = 'rosetta_300k_synthtiger' #folder name of model
# iteration = 'iter_70000' # model .pt file name

# model_path = os.path.join('trainer', 'saved_models',model)
# reader = easyocr.Reader(lang_list = ['en','th'],
#                     model_storage_directory = model_path,
#                     user_network_directory = os.path.join('my_model','user_network'),
#                     recog_network  = iteration,
#                     config_path = 'hippo_300k_synthtiger', #configuration file name use to get character_list only  (so able to use for every model)
#                     gpu = True)

##### Prediction with pth model

run this cell and edit config in deployment_config.py to deploy

In [ ]:
# # filepaths = glob.glob(os.path.join('trainer/all_data/testing/testing','*.jpg'))
# # filepaths = sorted(filepaths, key= lambda x: int(re.split(r'[/\\.]',x)[-2]))
# # # random_file_path = filepaths[10]
# # random_file_path = random.sample(filepaths,1)[0]
# random_file_path = '10.jpg'
# img = plt.imread(random_file_path)
# # print (f'image befor reader: {img[0][0]}')
# start = time.time()
# result = reader.recognize(random_file_path)
# end = time.time()
# print(f'time: {end - start}')
# print(result)

##### prediction with onnx model

In [ ]:
import matplotlib.pyplot as plt
from torchvision import transforms
from easyocr.utils import reformat_input
from easyocr.utils import get_image_list
from easyocr.recognition import AlignCollate
from easyocr.recognition import ListDataset
import torch.nn.functional as F
from easyocr.utils import CTCLabelConverter
import onnxruntime

In [ ]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy()if tensor.requires_grad else tensor.cpu().numpy()
def custom_mean(x):
    return x.prod()**(2.0/np.sqrt(len(x)))

In [ ]:
def onnx_recognize(img_path, ort_session, converter, device = torch.device("cuda")):
    # start = time.time()
    img, img_cv_grey = reformat_input(img_path)
    y_max, x_max = img_cv_grey.shape
    horizontal_list = [[0, x_max, 0, y_max]]
    for bbox in horizontal_list:
                    h_list = [bbox]
                    f_list = []
                    image_list, max_width = get_image_list(h_list, f_list, img_cv_grey, model_height = 64)

    img_list = [item[1] for item in image_list]

    AlignCollate_normal = AlignCollate(imgH=64, imgW=600, keep_ratio_with_pad=True)
    test_data = ListDataset(img_list)
    test_loader = torch.utils.data.DataLoader(
            test_data, batch_size=1, shuffle=False,
            num_workers=int(0), collate_fn=AlignCollate_normal, pin_memory=True)
    
    image_tensors = next(iter(test_loader))
    batch_size = image_tensors.size(0)
    image = image_tensors.to(device)
    # end = time.time()
    # print(f'pre-process : {end - start}')

    # start = time.time()
    ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(image)}
    ort_outs = ort_session.run(None, ort_inputs)
    preds = torch.from_numpy(ort_outs[0])
    # end = time.time()
    # print(f'prediction : {end - start}')

    ignore_idx = []
    start = time.time()
    # Select max probabilty (greedy decoding) then decode index to character
    preds_size = torch.IntTensor([preds.size(1)] * batch_size)

    ######## filter ignore_char, rebalance
    preds_prob = F.softmax(preds, dim=2)
    preds_prob = preds_prob.cpu().detach().numpy()
    preds_prob[:,:,ignore_idx] = 0.
    pred_norm = preds_prob.sum(axis=2)
    preds_prob = preds_prob/np.expand_dims(pred_norm, axis=-1)
    preds_prob = torch.from_numpy(preds_prob).float().to(device)
    result = []

    # end = time.time()
    # print(f'preds_prob : {end - start}')
    # decoder
    decoder = 'greedy'
    if decoder == 'greedy':
        # Select max probabilty (greedy decoding) then decode index to character
        _, preds_index = preds_prob.max(2)
        preds_index = preds_index.view(-1)
        preds_str = converter.decode_greedy(preds_index.data.cpu().detach().numpy(), preds_size.data)
    elif decoder == 'beamsearch':
        k = preds_prob.cpu().detach().numpy()
        preds_str = converter.decode_beamsearch(k, beamWidth=5)
    elif decoder == 'wordbeamsearch':
        k = preds_prob.cpu().detach().numpy()
        preds_str = converter.decode_wordbeamsearch(k, beamWidth=5)

    # end = time.time()
    # print(f'decoder : {end - start}')
    preds_prob = preds_prob.cpu().detach().numpy()
    values = preds_prob.max(axis=2)
    indices = preds_prob.argmax(axis=2)
    preds_max_prob = []
    for v,i in zip(values, indices):
        max_probs = v[i!=0]
        if len(max_probs)>0:
            preds_max_prob.append(max_probs)
        else:
            preds_max_prob.append(np.array([0]))

    for pred, pred_max_prob in zip(preds_str, preds_max_prob):
        confidence_score = custom_mean(pred_max_prob)
        result.append([pred, confidence_score])
    # end = time.time()
    # print(f'post-process : {end - start}')
    return result

load onnx model

In [ ]:
ort_session = onnxruntime.InferenceSession("rosetta_hippo_300k_bw_recognitionModel.onnx", providers=['CUDAExecutionProvider'])
ort_session.get_providers()
ort_session.set_providers(['CUDAExecutionProvider'])

predict with onnx

In [ ]:
import yaml
file_path = '/home/EasyOCR/my_model/user_network/hippo_300k_synthtiger.yaml'
with open(file_path, 'r', encoding="utf8") as stream:
        opt = yaml.safe_load(stream)

In [ ]:
character = opt['character_list']
separator_list = {}
dict_list = {}
converter = CTCLabelConverter(character, separator_list, dict_list)

In [ ]:
random_file_path = '10.jpg'
# random_file_path = filepaths[0]
start = time.time()
result = onnx_recognize(random_file_path, ort_session, converter)

end = time.time()
print(f'time: {end - start}')
print(result)

testing with 1000 dataset

In [ ]:
filepaths = glob.glob(os.path.join('trainer/all_data/testing/testing','*.jpg'))
filepaths = sorted(filepaths, key= lambda x: int(re.split(r'[/\\.]',x)[-2]))
label_path = 'trainer/all_data/testing/testing/label.txt'
with open(label_path, encoding='utf8') as file:
    label = np.loadtxt(file,dtype=str)
start = time.time()
pred_list = []
gt = [] # ground truth
name_img = []

so = onnxruntime.SessionOptions()
so.graph_optimization_level = onnxruntime.GraphOptimizationLevel.ORT_ENABLE_ALL
ort_session = onnxruntime.InferenceSession("rosetta_hippo_300k_bw_recognitionModel.onnx", providers=['CUDAExecutionProvider'], sess_options = so)
ort_session.set_providers(['CUDAExecutionProvider'])

# ort_session = onnxruntime.InferenceSession("crnn_recognitionModel.onnx", providers=['TensorrtExecutionProvider', 'CPUExecutionProvider'])
print(f'get_providers {ort_session.get_providers()} ')
for i_img in tqdm(range(len(filepaths))):
    label_img = label[i_img][1]
    name_img.append(label[i_img][0])
    result = onnx_recognize(filepaths[i_img], ort_session, converter)
    
    try:
        pred_list.append([result[0][0]])
    except:
        pred_list.append('')
    gt.append(label_img)


end = time.time()
print(f'time: {end - start}')


Evaludate

In [ ]:
tmp = name_img.copy()
output, false_list = eval_ocr_metric(pred_list, gt, tmp)
# output, false_list = eval_ocr_metric(pred, gt, name_img)
for k,v in output.items():
    print(f'{k}: {v}')

In [ ]:
plt.rcParams['xtick.labelsize'] = 20.0
plt.rcParams['ytick.labelsize'] = 20.0
columns = 3
rows = 2
fig = plt.figure(figsize=(15, 15))
ax = []
all_pred = []
filepaths = glob.glob(os.path.join('trainer/all_data/testing/testing','*.jpg'))
# filepaths = glob.glob(os.path.join('trainer/all_data/training/training','*.jpg'))
filepaths = sorted(filepaths, key= lambda x: int(re.split(r'[/\\.]',x)[-2]))
for i in tqdm(range(columns*rows)):
    # random_file_name = random.sample(os.listdir(os.path.join('trainer/all_data/testing/testing')),1)[0]
    # random_file_path = os.path.join('trainer/all_data/testing/testing',random_file_name)
    #random_file_name = random.sample(os.listdir(os.path.join('trainer/all_data/testing/testing')),1)[0]

    random_file_name = random.sample(filepaths,1)[0]
    random_file_path = random_file_name

    so = onnxruntime.SessionOptions()
    so.graph_optimization_level = onnxruntime.GraphOptimizationLevel.ORT_ENABLE_ALL
    ort_session = onnxruntime.InferenceSession("rosetta_recognitionModel.onnx", providers=['CUDAExecutionProvider'], sess_options = so)
    ort_session.set_providers(['CUDAExecutionProvider'])

    img = plt.imread(random_file_path)
    result = onnx_recognize(random_file_path, ort_session, converter)
    ax.append(fig.add_subplot(rows, columns, i+1) )
    # boxs = []
    txt = []
    for idx,i in enumerate(result):
        # boxs.append(i[0])
        txt.append(i[0])
    all_pred.append(txt)
    plt.axis('off')
    if len(result) > 0:
        ax[-1].set_title(f'{idx}: {txt} ', fontsize=15) 
        # for i in boxs:
        #         pts = np.array(i, np.int32)
        #         pts = pts.reshape((-1,1,2))
        #         cv.polylines(img,[pts],True,(0,0,0))
    else:
        ax[-1].set_title(f'file: {random_file_name} No result', fontsize=15)
    plt.imshow(img)
fig.tight_layout() 
plt.show()
print(all_pred)

In [ ]:
plt.rcParams['xtick.labelsize'] = 20.0
plt.rcParams['ytick.labelsize'] = 20.0
columns = 3
rows = 1
fig = plt.figure(figsize=(15, 15))
ax = []
all_pred = []
for i in tqdm(range(columns*rows)):
    # random_file_name = random.sample(os.listdir(os.path.join('trainer/all_data/testing/testing')),1)[0]
    # random_file_path = os.path.join('trainer/all_data/testing/testing',random_file_name)
    #random_file_name = random.sample(os.listdir(os.path.join('trainer/all_data/testing/testing')),1)[0]

    random_file_name = random.sample(false_list,1)[0]
    random_file_name = random_file_name[0]
    random_file_path = os.path.join('trainer/all_data/testing/testing',random_file_name)

    so = onnxruntime.SessionOptions()
    so.graph_optimization_level = onnxruntime.GraphOptimizationLevel.ORT_ENABLE_ALL
    ort_session = onnxruntime.InferenceSession("rosetta_recognitionModel.onnx", providers=['CUDAExecutionProvider'], sess_options = so)
    ort_session.set_providers(['CUDAExecutionProvider'])

    img = plt.imread(random_file_path)
    result = onnx_recognize(random_file_path, ort_session, converter)
    ax.append(fig.add_subplot(rows, columns, i+1) )
    # boxs = []
    txt = []
    for idx,i in enumerate(result):
        # boxs.append(i[0])
        txt.append(i[0])
    all_pred.append(txt)
    plt.axis('off')
    if len(result) > 0:
        ax[-1].set_title(f'file: {random_file_name} {idx}: {txt} ', fontsize=15) 
        # for i in boxs:
        #         pts = np.array(i, np.int32)
        #         pts = pts.reshape((-1,1,2))
        #         cv.polylines(img,[pts],True,(0,0,0))
    else:
        ax[-1].set_title(f'file: {random_file_name} No result', fontsize=15)
    plt.imshow(img)
fig.tight_layout() 
plt.show()
print(all_pred)

## bw

In [ ]:
filepaths = glob.glob(os.path.join('trainer/all_data/testing/test_bw','*.jpg'))
filepaths = sorted(filepaths, key= lambda x: int(re.split(r'[/\\.]',x)[-2]))
label_path = 'trainer/all_data/testing/test_bw/label.txt'
label= []
with open(label_path, encoding='utf8') as file:
    lines = file.readlines()
    for line in lines:
        label.append(eval(line))
start = time.time()
# filepaths = filepaths[:3]
pred = []
gt = [] # ground truth
name_img = []
so = onnxruntime.SessionOptions()
so.graph_optimization_level = onnxruntime.GraphOptimizationLevel.ORT_ENABLE_ALL
ort_session = onnxruntime.InferenceSession("rosetta_hippo_300k_bw_recognitionModel.onnx", providers=['CUDAExecutionProvider'], sess_options = so)
ort_session.set_providers(['CUDAExecutionProvider'])
print(len(filepaths), len(label))
for i in tqdm(range(len(filepaths))):
    label_img = label[i]['text']
    name_img.append(label[i]['filename'])
    result = onnx_recognize(filepaths[i], ort_session, converter)
    try:
        pred.append([result[0][0]])
    except:
        pred.append('')
    gt.append(label_img)
end = time.time()
print(f'time: {end - start}')

In [ ]:
tmp = name_img.copy()
output, false_list = eval_ocr_metric(pred, gt, tmp)
# output, false_list = eval_ocr_metric(pred, gt, name_img)
for k,v in output.items():
    print(f'{k}: {v}')